In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.shape

(57650, 4)

In [5]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

Dropping link column

In [6]:
df = df.sample(5000).drop("link", axis =1).reset_index(drop=True)

In [7]:
df.head()

,artist,song,text
0,Chris Brown,Fools With You,"White Flag Hangs Up, I Aint Got No Room For Th..."
1,Pearl Jam,Corduroy,The waiting drove me mad... you're finally her...
2,Veruca Salt,With David Bowie,\r\n \r\n \r\n \r\n \r\nwill it ever com...
3,Green Day,Stop When The Red Lights Flash,Uncomfortable silence \r\nI'll kill it if you...
4,NOFX,Professional Crastination,Can't you see how easy it becomes? \r\nProcra...


In [8]:
df['text'][0]

"White Flag Hangs Up, I Aint Got No Room For Them (No)  \r\nIf Love Was A Crime, I'd Be Doing Way More Then Life  \r\nGirl I'm Through All I See Is You, Your My Heart Girl I Fools With You.  \r\n  \r\n(Chorus)  \r\nTell Me When, Tell Me Where,  \r\nWhatever You Want Baby I'll Do It All For You  \r\nKinda Love Got Me Sprung  \r\nI'm In It And I don't Give A Damn  \r\nAll These Girls On My Head Like A Fresh Cut  \r\nTry'na Get Yo Spot But They Gets No Love  \r\nKinda Love Got Me Sprung  \r\nBaby For The Things You Do I Fools With You  \r\n  \r\n(Second Verse)  \r\nYou Are what's Up, Baby I Aint Got No Reason To Lie  \r\nGirl I'm Feelin Locked In But Not Stuck, But You Aint Never Always The Type  \r\nGirl I'm Through All I See Is You  \r\nYour My Heart Girl I Fools With You  \r\n  \r\n(Chorus)  \r\nTell Me When, Tell Me Where,  \r\nWhatever You Want Baby I'll Do It All For You  \r\nKinda Love Got Me Sprung  \r\nI'm In It And I don't Give A Damn  \r\nAll These Girls On My Head Like A Fresh

In [9]:
#df = df.sample(5000).reset_index()

In [10]:
df.shape

(5000, 3)

Cleaning

Text Cleaning/ Text Pre processing

In [11]:
df['text'] = df['text'].str.lower().replace(r'^\W\s',' ').replace(r'\n',' ', regex=True).replace(r'\r',' ', regex=True)

In [12]:
df.shape

(5000, 3)

In [13]:
df['text'].head()

0    white flag hangs up, i aint got no room for th...
1    the waiting drove me mad... you're finally her...
2                        will it ever come?        ...
3    uncomfortable silence    i'll kill it if you w...
4    can't you see how easy it becomes?    procrast...
Name: text, dtype: object

In [14]:
df.shape

(5000, 3)

In [15]:
import nltk

In [16]:
from nltk.stem.porter import PorterStemmer

In [17]:
stemmer = PorterStemmer() 

In [18]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [19]:
token("beauty is beautiful, beauti")

'beauti is beauti , beauti'

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
df['text'].apply(lambda x: token(x))

0       white flag hang up , i aint got no room for th...
1       the wait drove me mad ... you 're final here a...
2       will it ever come ? like it did for you like i...
3       uncomfort silenc i 'll kill it if you want me ...
4       ca n't you see how easi it becom ? procrastin ...
                              ...                        
4995    how mani time i say the word again and again a...
4996    my fine feather friend with your cute littl pr...
4997    it wa a hot hot day , middl of may , saw my ba...
4998    [ j. gill ] tumbl dice seven on 'em , babi yea...
4999    flood of tear and door slam stamp feet across ...
Name: text, Length: 5000, dtype: object

In [22]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [23]:
matrix = tfid.fit_transform(df['text'])

In [24]:
matrix

<5000x24221 sparse matrix of type '<class 'numpy.float64'>'
	with 268963 stored elements in Compressed Sparse Row format>

In [25]:
similar = cosine_similarity(matrix)

In [26]:
similar[1]

array([0.07698965, 1.        , 0.06999501, ..., 0.03112723, 0.03903387,
       0.03388528])

In [29]:
df[df['song']=="Stop When The Red Lights Flash"].index[0]

3

In [30]:
enumerate[1,1,1 ]

enumerate[1, 1, 1]

In [31]:
def recommender(song_df):
    idx = df[df['song']==song_df].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse = True, key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]: 
        song.append(df.iloc[s_id[0]].song)
    return song

In [33]:
recommender("Stop When The Red Lights Flash")

['Remember December',
 'Surrender To Me',
 'Love Me Tonight',
 'Gonna Make You Mine']

Saving the model

In [34]:
import pickle

In [35]:
pickle.dump(similar, open("similarity.pkl", "wb"))

In [36]:
pickle.dump(df, open("df.pkl", "wb"))